## Sitting Posture Inference Server

### Load the model

In [ ]:
import sys
import os
import torch

# Add the directory to sys.path
model_dir = "../models/2024-11-24_00-04-05"
sys.path.append(model_dir)

# Import the MLP class
from model import MLP

### Get test data

In [5]:
import pandas as pd
import numpy as np

# Load the dataset
data = pd.read_csv("../../datasets/vectors/augmented_xy_filtered_keypoints_vectors_mediapipe.csv")

# Extract the first row
row = data.iloc[0]

# Separate features and label
features = row.drop("class").values.astype(np.float32)  # Ensure float32 for PyTorch
true_label = row["class"]

# Load the scaler (use the same scaler from training)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_features = data.drop(columns=["class"]).values  # Exclude the 'class' column
scaler.fit(data_features)  # Fit scaler on the entire dataset (or use saved scaler)
features = scaler.transform([features])  # Normalize the feature vector

# Convert to tensor
features_tensor = torch.tensor(features, dtype=torch.float32)

### Recreate the model

In [7]:
input_size = features_tensor.shape[1]  # Number of features
num_classes = len(data["class"].unique())  # Number of unique classes
model = MLP(input_size=input_size, num_classes=num_classes)

# Load the saved model weights
model.load_state_dict(torch.load("../models/2024-11-24_00-04-05/epochs_50_lr_1e-03_acc_76.pth"))
model.eval()  # Set the model to evaluation mode

C:\Users\User\AppData\Local\Temp\ipykernel_72736\959964167.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("../models/2024-11-24_00-04-0

MLP(
  (model): Sequential(
    (0): Linear(in_features=20, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=4, bias=True)
  )
)

### Do Prediction

In [11]:
# Predict
with torch.no_grad():
    output = model(features_tensor)
    _, predicted_class = torch.max(output, 1)

# Print the results
class_labels = ["crossed_legs", "proper", "slouching", "reclining"]  # Update based on your class order
print(f"True label: {true_label}")
print(f"Predicted label: {predicted_class.item()} ({class_labels[predicted_class.item()]})")

if true_label == class_labels[predicted_class.item()]:
    print("Correct prediction!")

True label: crossed_legs
Predicted label: 0 (crossed_legs)
Correct prediction!
